In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns 
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import FixedLocator
import statsmodels.api as sm

In [2]:
folder_path = 'data'
dataframes = []


for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_excel(file_path)
        
        # Calculate mean and standard deviation of spaceRT
        # Calculate 3*std for spaceRT and add as a new column we do this for knowing the spaceRT outliers and remove them in
        # calculating spaceRT cause these are rest somehow 
        # the spaceRT_2sd is obviously same in all columns cause it's based on all the spaceRTs. I just wanted to have everything
        # in the dataframe
        mean_spaceRT = df['spaceRT'].mean()
        std_spaceRT = df['spaceRT'].std()
        df['spaceRT_2sd'] = mean_spaceRT + 2 * std_spaceRT
        

        dataframes.append(df)

dataframes[5].head(3)

,arrowRT,distribution,interTrialInterval,outcome,myCard,yourCard,spaceRT,totalReward,trialIndex,trialType,choice,block,timeoutRepeat,spaceRT_2sd
0,na,uniform,879.0,na,4.0,2.0,10433.0,na,0.0,timeout,na,1.0,0.0,5115.974575
1,820.0,uniform,996.0,win,4.0,6.0,2306.0,10.5,1.0,response,arrowdown,1.0,0.0,5115.974575
2,397.0,uniform,896.0,win,8.0,3.0,2601.0,11.0,2.0,response,arrowup,1.0,0.0,5115.974575


In [3]:
for df in dataframes:
    df['block_type'] = None

    df.loc[df['block'] == 1, 'block_type'] = 'uniform'     # Block 1 is uni
    df.loc[df['block'] == 4, 'block_type'] = 'mix'     # Block 4 is mix

    # For blocks 2 and 3, set based on distribution
    df.loc[(df['block'] == 2) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 2) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    
dataframes[0].head(3)   

,arrowRT,distribution,interTrialInterval,outcome,myCard,yourCard,spaceRT,totalReward,trialIndex,trialType,choice,block,timeoutRepeat,spaceRT_2sd,block_type
0,570.0,uniform,831.0,lose,5.0,2.0,2209.0,9.5,0.0,response,arrowdown,1.0,0.0,33269.609946,uniform
1,1162.0,uniform,901.0,lose,4.0,3.0,5755.0,9.0,1.0,response,arrowdown,1.0,0.0,33269.609946,uniform
2,355.0,uniform,939.0,win,4.0,6.0,1209.0,9.5,2.0,response,arrowdown,1.0,0.0,33269.609946,uniform


In [4]:
# 1) Define the risk dictionaries for each deck type
risk_uniform = {
    1: 0.0,
    2: 0.125,
    3: 0.25,
    4: 0.375,
    5: 0.50,
    6: 0.375,
    7: 0.25,
    8: 0.125,
    9: 0.0
}

risk_low = {
    1: 0.000,
    2: 0.243,
    3: 0.447,
    4: 0.385,
    5: 0.250,
    6: 0.146,
    7: 0.071,
    8: 0.023,
    9: 0.000
}

risk_high = {
    1: 0.000,
    2: 0.023,
    3: 0.071,
    4: 0.146,
    5: 0.250,
    6: 0.385,
    7: 0.447,
    8: 0.243,
    9: 0.000
}

# 2) Wrap them in one master dictionary keyed by distribution
risk_map = {
    'uniform': risk_uniform,
    'low':     risk_low,
    'high':    risk_high
    # If you have a 'mix' condition, decide how to handle or skip it
}

# 3) For each DataFrame in your list, create a 'risk' column
for df in dataframes:
    df['risk'] = df.apply(
        lambda row: risk_map.get(row['distribution'], {}).get(row['myCard'], np.nan),
        axis=1
    )
    

dataframes[1].head(3)


,arrowRT,distribution,interTrialInterval,outcome,myCard,yourCard,spaceRT,totalReward,trialIndex,trialType,choice,block,timeoutRepeat,spaceRT_2sd,block_type,risk
0,565.0,uniform,955.0,lose,1.0,2.0,1707.0,9.5,0.0,response,arrowup,1.0,0.0,32779.850145,uniform,0.00
1,338.0,uniform,825.0,win,5.0,7.0,2356.0,10.0,1.0,response,arrowdown,1.0,0.0,32779.850145,uniform,0.50
2,727.0,uniform,785.0,win,3.0,4.0,1469.0,10.5,2.0,response,arrowdown,1.0,0.0,32779.850145,uniform,0.25


In [5]:
output_folder = 'data_risk_added'

for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):
        # Read the Excel file
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_excel(file_path)
        
        # Compute the new 'risk' column
        def compute_risk(row):
            dist = row['distribution']
            card = row['myCard']
            return risk_map.get(dist, {}).get(card, np.nan)
        
        df['risk'] = df.apply(compute_risk, axis=1)

        # (Optional) do any other modifications you need here...
        
        # Save the updated DataFrame with the same filename to data_risk_added
        out_path = os.path.join(output_folder, file_name)
        df.to_excel(out_path, index=False)

print("All files processed and saved with 'risk' column in", output_folder)

All files processed and saved with 'risk' column in data_risk_added
